In [1]:
from vk_tools import *

In [4]:
import os
os.chdir(r"C:\Users\ATsareva\Desktop\LARP")
import pandas as pd
import time
import networkx as nx

In [5]:
time_now = lambda pattern: time.strftime(pattern, time.localtime())

In [6]:
def get_members_for_now(group_id):
    all_members = get_all_members(group_id)
    now = time_now("%Y-%m-%d %H:%M:%S")
    df = pd.DataFrame.from_dict(all_members)
    df['first_seen'] = now
    df.bdate = pd.to_datetime(df.bdate, errors='coerce')
    df = get_age_from_profile(df)
    df.to_excel('members_group'+str(group_id)+" "+now.replace(":", "")+'.xlsx')
    return df

def get_wall_for_now(group_id):
    owner_id = group_id*-1
    wall = get_all_wall(owner_id)
    now = time_now("%Y-%m-%d %H:%M:%S")
    for post in wall:
        post['users_who_liked'] = str(get_all_likes(owner_id, post['id']))
    wall_df = pd.DataFrame.from_dict(wall)
    wall_df.date = pd.to_datetime(wall_df.date, errors='coerce')
    wall_df.to_excel('wall_group'+str(group_id)+" "+now.replace(":", "")+'.xlsx')
    return wall_df

def users_friends(users_id):
    all_users = []
    n=1
    #list structure: [user_id, [all_friends_ids]]
    for user in users_id:
        try:
            friends = get_all_friends(str(user))
            list_of_friends = friends['id'].values
            one_user = [user, list_of_friends]
            all_users.append(one_user)
        except:
            print('Не получить данные для ', user)
            pass
        print('Обработано ', n, ' из ', len(users_id))
        n+=1
    return all_users

def add_intersections_egde(id_with_members, all_users, G):
    node = id_with_members[0]
    members = id_with_members[1]
    for user in all_users:
        if user[0] != node:
            if user[0] in members:
                G.add_edge(node, user[0])

In [19]:
group_id = 147852113
df = get_members_for_now(group_id)
print('Размер сообщества: ', len(df))

Размер сообщества:  263


In [6]:
wall_df = get_wall_for_now(group_id)

In [20]:
users_id = df.id.values
all_users = users_friends(users_id)

Обработано  1  из  263
Обработано  2  из  263
Обработано  3  из  263
Обработано  4  из  263
Обработано  5  из  263
Обработано  6  из  263
Обработано  7  из  263
Обработано  8  из  263
Обработано  9  из  263
Обработано  10  из  263
Обработано  11  из  263
Обработано  12  из  263
Обработано  13  из  263
Обработано  14  из  263
Обработано  15  из  263
Обработано  16  из  263
Обработано  17  из  263
Обработано  18  из  263
Обработано  19  из  263
Обработано  20  из  263
Обработано  21  из  263
Обработано  22  из  263
Обработано  23  из  263
Обработано  24  из  263
Обработано  25  из  263
Обработано  26  из  263
Обработано  27  из  263
Обработано  28  из  263
Обработано  29  из  263
Обработано  30  из  263
Обработано  31  из  263
Обработано  32  из  263
Обработано  33  из  263
Обработано  34  из  263
Обработано  35  из  263
Обработано  36  из  263
Обработано  37  из  263
Обработано  38  из  263
Обработано  39  из  263
Обработано  40  из  263
Обработано  41  из  263
Обработано  42  из  263
О

In [21]:
G=nx.DiGraph()

In [22]:
df.fillna('', inplace=True)
for user in all_users:
    city = df[df.id==user[0]]['city'].values[0]
    country = df[df.id==user[0]]['country'].values[0]
    name = df[df.id==user[0]]['first_name'].values[0] + " " + df[df.id==user[0]]['last_name'].values[0]
    university_name = df[df.id==user[0]]['university_name'].values[0]
    if df[df.id==user[0]]['sex'].values[0] == 1:
        sex = 'женский'
    else:
        sex = 'мужской'
    age = df[df.id==user[0]]['age'].values[0]
    G.add_node(user[0], city=city, country=country, name=name, 
               university_name = university_name, 
               age=age, sex=sex)

In [23]:
for user in all_users:
    add_intersections_egde(user, all_users, G)

In [24]:
now = time_now("%Y-%m-%d %H:%M:%S")
nx.write_graphml(G, 'group'+str(group_id)+" "+now.replace(":", "")+".graphml")